<a href="https://colab.research.google.com/github/Bl0zkp/ETMACH/blob/main/Examen_Machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Paso 1**   Orden para poder encontrar las carpetas


In [ ]:
import os, sys
from pathlib import Path

PROJECT_ROOT = Path("/content/home-credit-risk")

os.chdir(PROJECT_ROOT)                 # nos movemos a la raíz del proyecto
sys.path.insert(0, str(PROJECT_ROOT))  # agregamos la raíz al PYTHONPATH

print("CWD:", Path.cwd())
print("PYTHONPATH[0]:", sys.path[0])


CWD: /content/home-credit-risk
PYTHONPATH[0]: /content/home-credit-risk


# **si la carpeta data existe no ejecutar**

In [ ]:
%cd /content/home-credit-risk
!mkdir -p data/raw
!ls data


/content
raw


In [ ]:
!cp /content/*.parquet /content/home-credit-risk/data/raw/
!ls /content/home-credit-risk/data/raw


application_.parquet	     HomeCredit_columns_description.parquet
bureau_balance.parquet	     installments_payments.parquet
bureau.parquet		     POS_CASH_balance.parquet
credit_card_balance.parquet  previous_application.parquet


# **Paso 2**   Se ejecuta el build data set para poder crear el json y la union de las tablas **

In [ ]:
%cd /content/home-credit-risk
!python 02_data_preparation/build_dataset.py


/content
OK: dataset guardado en data/processed/dataset.parquet
OK: schema guardado en artifacts/feature_schema.json


# **Paso 3** Se ejecuta el train para las metricas de validaciones

In [ ]:
!python 03_modeling/train.py


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
OK: modelo guardado en artifacts/model.joblib
Métricas validación: {'roc_auc': 0.6814058378938038, 'pr_auc': 0.15741549490185353}


# **Paso 4** Se Ejecuta el evaluate para saber la distribucion de deciciones y se interpretan metricas

In [ ]:
!python 04_evaluation/evaluate.py

OK: reporte guardado en artifacts/evaluation_report.json
{'roc_auc': 0.6816834032164718, 'pr_auc': 0.15662159190885433, 'approve_threshold': 0.2, 'reject_threshold': 0.5, 'confusion_matrix_at_reject_threshold': [[176432, 106254], [8758, 16067]], 'decision_counts': {'REVISIÓN MANUAL': 173091, 'RECHAZAR': 122321, 'APROBAR': 12099}}



# **Paso 5**  Ejecucion de la api como servidor

In [ ]:
!uvicorn 05_deployment.app:app --reload

INFO:     Will watch for changes in these directories: ['/content/home-credit-risk']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [40379] using StatReload
INFO:     Started server process [40381]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [40381]
INFO:     Stopping reloader process [40379]


# **Inicia el servidor en segundo plano para poder hacer consultas**

In [ ]:
!pkill -f uvicorn || true
!setsid python3 -m uvicorn 05_deployment.app:app --host 127.0.0.1 --port 8000 > api.log 2>&1 < /dev/null &

^C


# **Lista de los procesos que estan ocurriendo**

In [ ]:
!ps aux | grep uvicorn

root       47720 70.0  0.9 387060 130556 ?       Rsl  01:50   0:01 python3 -m uvicorn 05_deployment.app:app --host 127.0.0.1 --port 8000
root       47729  0.0  0.0   7372  3340 ?        S    01:50   0:00 /bin/bash -c ps aux | grep uvicorn
root       47731  0.0  0.0   6480  2384 ?        S    01:50   0:00 grep uvicorn


# **matar el procesos de servidor en segundo plano**

In [ ]:
!kill -9 -13879 #aca va el nro que aparece en el ouput anterior

# **Pruebas de salud**

In [ ]:
!curl http://127.0.0.1:8000/health

{"status":"ok"}

# **Prueba de Peticion**

In [ ]:
import requests

url = "http://127.0.0.1:8000/evaluate_risk"

payload = {
    "data": {
        "AMT_INCOME_TOTAL": 150000, #ingreso total anual
        "DAYS_BIRTH": -12000,  #edad en dias negativos
        "DAYS_EMPLOYED": -2000 #antiguedad laboral en negativos
    }
}

response = requests.post(url, json=payload)
print(response.status_code)
print(response.json())


200
{'probabilidad_incumplimiento': 0.4869845494335178, 'decision_sugerida': 'REVISIÓN MANUAL', 'umbral_aprobar': 0.2, 'umbral_rechazar': 0.5}


In [ ]:
%cd /content/home-credit-risk
!ls

/content
02_data_preparation  03_modeling  04_evaluation  05_deployment	Artifacts
